In [1]:
import mysql.connector
import re
import random

def querymysql(query_command,dbconfig):
    db=mysql.connector.connect(host=dbconfig[0],
                           database=dbconfig[1],
                           user=dbcongfig[2],
                           password=dbconfig[3])
    cursor=db.cursor()
    cursor.execute(query_command)
    cursor.close()
    db.close()
    queryresult=cursor.fetchall()
    return queryresult

userid=14
dbconfig=['localhost','lccgame','host','host']

In [56]:
#--------------------------------------------------------------------------------------
# fun: get all the agent ID in backup_scalsc_states
# input: none
# outout:['p12','p14',...]
#--------------------------------------------------------------------------------------
def getallinterid():
    db=mysql.connector.connect(host="localhost",
                           database='lccgame',
                           user='host',
                           password='host')
    cursor=db.cursor()
    #cursor.execute("SELECT COMM_ID FROM backup_scalsc_states ")
    """if tablename=='backup_scalsc_states':
        querycomd="SELECT COMM_ID FROM %s"%tablename
    else:
        querycomd="SELECT interid FROM %s"%tablename"""
    querycomd="SELECT interid FROM playerinfo"
    cursor.execute(querycomd)
    db.close()
    queryresult=cursor.fetchall()
    newresult=list(set(queryresult))#delete dupilcate elements
    allinterID=[]
    for i in range(len(newresult)):
        allinterID=allinterID+[newresult[i][0]]
    return allinterID

#--------------------------------------------------------------------------------------
# fun: get all the agent ID in backup_scalsc_states
# input: none
# outout:['p12','p14',...]
#--------------------------------------------------------------------------------------
def getallagentid():
    db=mysql.connector.connect(host="localhost",
                           database='lccgame',
                           user='host',
                           password='host')
    cursor=db.cursor()
    #cursor.execute("SELECT COMM_ID FROM backup_scalsc_states ")
    """if tablename=='backup_scalsc_states':
        querycomd="SELECT COMM_ID FROM %s"%tablename
    else:
        querycomd="SELECT interid FROM %s"%tablename"""
    querycomd="SELECT userid FROM playerinfo"
    cursor.execute(querycomd)
    db.close()
    queryresult=cursor.fetchall()
    newresult=list(set(queryresult))#delete dupilcate elements
    allagentID=[]
    for i in range(len(newresult)):
        allagentID=allagentID+[newresult[i][0]]
    return allagentID

#--------------------------------------------------------------------------------------
# fun: get playerinfo by interid in playerinfo
# input: inter id
# outout: {'interid','playerrole','playerid'}
#--------------------------------------------------------------------------------------
def getplayerinfo_byinterid(interid):
    db=mysql.connector.connect(host="localhost",
                           database='lccgame',
                           user='host',
                           password='host')
    cursor=db.cursor()
    cursor.execute("SELECT userid FROM playerinfo WHERE interid=%r" %interid)
    playerid=cursor.fetchall()
    cursor.execute("SELECT playerrole FROM playerinfo WHERE interid=%r" %interid)
    playerrole=cursor.fetchall()
    db.close()
    playerinfo_list=[]
    for i in range(len(playerid)):
        playerinfo_list.append({
            'interid':interid,
            'playerid':playerid[i][0],
            'playerrole':playerrole[i][0]
        })
    return playerinfo_list

#--------------------------------------------------------------------------------------
# fun: get playerinfo by agentid in playerinfo
# input: inter id
# outout: {'interid','playerrole','playerid'}
#--------------------------------------------------------------------------------------
def getplayerinfo_byagentid(agentid):
    db=mysql.connector.connect(host="localhost",
                           database='lccgame',
                           user='host',
                           password='host')
    cursor=db.cursor()
    cursor.execute("SELECT interid FROM playerinfo WHERE userid=%r" %agentid)
    interid=cursor.fetchall()
    cursor.execute("SELECT playerrole FROM playerinfo WHERE userid=%r" %agentid)
    playerrole=cursor.fetchall()
    db.close()
    playerinfo_list=[]
    for i in range(len(interid)):
        playerinfo_list.append({
            'interid':interid[i][0],
            'playerid':agentid,
            'playerrole':playerrole[i][0]
        })
    return playerinfo_list


#--------------------------------------------------------------------------------------
# fun: get gamemsgs by interid in gamemsgs2
# input: interid
# outout: {'interid','playerrole','playerid'}
#--------------------------------------------------------------------------------------
def getgamemsgs_byinterid(interid):
    db=mysql.connector.connect(host="localhost",
                           database='lccgame',
                           user='host',
                           password='host')
    cursor=db.cursor()
    cursor.execute("SELECT * FROM gamemsgs WHERE interid='%s'"%interid)
    gamemsgsdata=cursor.fetchall()
    db.close()
    gamemsgs_list=[]
    for i in range(len(gamemsgsdata)):
        gamemsgs_list.append({
            'gameid':gamemsgsdata[i][0],
            'interid':gamemsgsdata[i][1],
            'protocol':gamemsgsdata[i][2],
            'msgsenderid':gamemsgsdata[i][3],
            'msgsenderrole':gamemsgsdata[i][4],
            'msgreceiverid':gamemsgsdata[i][5],
            'msgreceicerrole':gamemsgsdata[i][6],
            'msgbody':gamemsgsdata[i][7]
        })
    return gamemsgs_list

#------------------------------------------
# fun: Get message details
# input: m(x)
# output:{'mname':...,'mvars':...}
#------------------------------------------
def getmessage_detail(messagedata):
    p_message=re.compile(r'\s*(?P<mname>\w*)\((?P<mvars>.*)\s*\)')
    m_message=re.match(p_message,messagedata)
    return{'messagename':m_message.group('mname'),'messagecontent':m_message.group('mvars')}

In [57]:
allagentid=getallagentid()
allinterid=getallinterid()

# get playerinfo when the player acts as responder or trustee
p_playerinfo=getplayerinfo_byagentid('p12')


In [70]:
#-----------------------------------------------------------------
# get accept ratio and reject ratio for the player as responder
#-----------------------------------------------------------------

#get interid list as the responder
interidlist_responder=[]

for i in range(len(p_playerinfo)):
    if 'responder' in p_playerinfo[i]['playerrole']:
        if not p_playerinfo[i]['interid'] in interidlist_responder:
            interidlist_responder.append(p_playerinfo[i]['interid'])
            
#get game msgs as responder
gamemsgslist_responder=[]

for i in range(len(interidlist_responder)):
    gamemsgs=getgamemsgs_byinterid(interidlist_responder[i])
    gamemsgslist_responder.append(gamemsgs)

#get reply information as responder
replydata_responder=[]

for i in range(len(gamemsgslist_responder)):
    for gamemsgs in gamemsgslist_responder[i]:
        if 'p12' in gamemsgs['msgsenderid'] and 'acceptornot' in gamemsgs['msgbody']:
            responder_choice=getmessage_detail(gamemsgs['msgbody'])['messagecontent']
            agent_role=gamemsgs['msgsenderrole']
            interid=gamemsgs['interid']
        if 'p12' in gamemsgs['msgreceiverid'] and 'offernum' in gamemsgs['msgbody']:
            responder_offer=getmessage_detail(gamemsgs['msgbody'])['messagecontent']
    replydata_responder.append([responder_choice,responder_offer,agent_role,interid])

# calculate accept ratio and reject ratio
acceptratio_list=[]
rejectratio_list=[]

p_responder_dec=re.compile(r'acceptornot\((?P<choice>\w*)\#(?P<offernum>\d*\.*\d*)\)')
for i in range(len(replydata_responder)):
    m_dec=p_responder_dec.search(replydata_responder[i][0])
    resp_choice=m_dec.group('choice')
    resp_offernum=m_dec.group('offernum')
    if resp_choice=='accept':
        try:
            acceptratio_list.append([float(resp_offernum)/10,replydata_responder[i][3]])
        except:
            print("the %dth can't be floated"%i)
    if resp_choice=='reject':
        try:
            rejectratio_list.append([float(resp_offernum)/10,replydata_responder[i][3]])
        except:
            print("the %dth can't be floated"%i)
   


"""print(len(acceptratio_list))
print(len(rejectratio_list))"""

43
68


In [104]:
#-----------------------------------------------------------------
# get accept repay ratio for the player as trustee
#-----------------------------------------------------------------

#get interid list as the trustee
interidlist_trustee=[]

for i in range(len(p_playerinfo)):
    if 'trustee' in p_playerinfo[i]['playerrole']:
        if not p_playerinfo[i]['interid'] in interidlist_trustee:
            interidlist_trustee.append(p_playerinfo[i]['interid'])

                       
#get game msgs as trustee
gamemsgslist_trustee=[]

for i in range(len(interidlist_trustee)):
    gamemsgs=getgamemsgs_byinterid(interidlist_trustee[i])
    gamemsgslist_trustee.append(gamemsgs)

repaydata_trustee=[]

for i in range(len(gamemsgslist_trustee)):
    for gamemsgs in gamemsgslist_trustee[i]:
        if 'p12' in gamemsgs['msgsenderid'] and 'repay' in gamemsgs['msgbody']:
            trustee_repay=getmessage_detail(gamemsgs['msgbody'])['messagecontent']
            agent_role=gamemsgs['msgsenderrole']
            interid=gamemsgs['interid']
        if 'p12' in gamemsgs['msgreceiverid'] and 'invest' in gamemsgs['msgbody']:
            trustee_offer=getmessage_detail(gamemsgs['msgbody'])['messagecontent']
    repaydata_trustee.append([trustee_repay,trustee_offer,agent_role,interid])



repayratio_list=[]


p_trustee_repay=re.compile(r'repay\((?P<choice>\d*\.*\d*)\#')
p_trustee_invest=re.compile(r'invest\((?P<choice>\d*\.*\d*)\#')
p_trustee_rate=re.compile(r'trustee\((?P<investrate>.*)\)')
for i in range(len(repaydata_trustee)):
    m_repay=p_trustee_repay.search(repaydata_trustee[i][0])
    m_invest=p_trustee_invest.search(repaydata_trustee[i][1])
    m_rate=p_trustee_rate.search(repaydata_trustee[i][2])
    trustee_repay=m_repay.group('choice')
    trustee_invest=m_invest.group('choice')
    trustee_rate=m_rate.group('investrate')
    try:
        if float(trustee_invest)!=0:
            repayratio_list.append([float(trustee_repay)/(float(trustee_rate)*float(trustee_invest)),repaydata_trustee[i][3]])
        else:
            repayratio_list.append([0,repaydata_trustee[i][3]])
    except:
        print("the %dth ratio can't be floated"%i)


trustee(3) 
 3


In [97]:
repaydata_trustee[0][2]

'trustee(3)'